In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
df = pd.read_json("lessons_pd.json")
df.head(1000)

,date,start,end,subject,class,teacher,classroom
0,12/05,790,870,Линейная алгебра и аналитическая геометрия,DHT-1-20,Елена Бурова,LAB1
1,12/05,790,870,Линейная алгебра и аналитическая геометрия,DHT-2-20,Елена Бурова,LAB1
2,12/05,790,870,Линейная алгебра и аналитическая геометрия,EHI-1-20,Елена Бурова,LAB1
3,12/05,790,870,Линейная алгебра и аналитическая геометрия,SEST-1-20,Елена Бурова,LAB1
4,12/05,790,870,Линейная алгебра и аналитическая геометрия,SFHT-1-20,Елена Бурова,LAB1
...,...,...,...,...,...,...,...
995,11/29,700,780,Иностранный язык,MIX-1-22,Наталья Цуканова,C5
996,11/29,700,780,Иностранный язык,SEST-3-22,Гульмира Молдомусаева,C8
997,11/29,700,780,Иностранный язык,MIX-1-22,Гульмира Молдомусаева,C8
998,11/29,790,870,География,SEST-3-22,Чолпон Джусупова,C5


In [14]:
import datetime
фдд_сфиы = all_cabs = set(df["classroom"].unique())-{None}
сфиы = cabs = {cab for cab in all_cabs if not(cab.isdigit()|(cab[0]=="G"))}
еуфсрукы = teachers = list(df["teacher"].unique())
def from_time(hours, minutes=None):
    if minutes is None:
        hours, minutes = hours.split(":")
    return int(hours)*60+int(minutes)
def to_time(time):
    return f"{time//60}:"+str(time%60).ljust(2, '0')
def now(df=df):
    today = datetime.datetime.now()
    return then(df, from_time(today.hour, today.minute))
def time_overlap(df, time=None):
    if isinstance(time, str):
        time = from_time(time)
    if isinstance(time, list):
        time = from_time(*time)
    if time is None:
        return now(df)
    return df[(df["start"]<=time) & (df["end"]>=time)]
def then(df, time, date=None):
    today = datetime.datetime.now()
    if date is None:
        date = str(today.month).rjust(2, "0")+"/"+str(today.day).rjust(2, "0")
    return time_overlap(df[(df["date"] == date)], time)
def overlap(df, fields):
    df = df.copy()
    try:
        df = time_overlap(df, fields["time"])
        del fields["time"]
    except KeyError:
        pass
    for key in fields:
        df = df[df[key].apply(lambda x: fields[key] in x if x else False)]
    return df
def where_will(df, teacher, date):
    return overlap(df, {"teacher": teacher})[df["date"] == date]
def where(df, teacher):
    today = datetime.datetime.now()
    return where_will(df, teacher, f"{today.day}/{today.month}")
def show(df=df):
    df = df.copy()

    new_df = pd.DataFrame(columns = df.keys())
    for entry in df.values:
        overlap = list(df[(df["date"]==entry[0]) & (df["start"]==entry[1]) & (df["classroom"]==entry[6])]["class"].values)
        repeat = new_df[(new_df["date"]==entry[0]) & (new_df["start"]==entry[1]) & (new_df["classroom"]==entry[6])].index
        if len(repeat) == 0:
            entry[4] = overlap
            new_df.loc[len(new_df.index)] = entry
    df = sorted_df(new_df)
    df["start"] = df["start"].apply(to_time)
    df["end"] = df["end"].apply(to_time)
    return df
def will_free(df, time, date=None):
    return cabs-set(then(df, time, date)["classroom"])
def free(df=df):
    return cabs-set(now(df)["classroom"])
def sorted_df(df=df):
    df = df.copy()
    return df.sort_values(["date", "start", "end"])
def today(df=df):
    now = datetime.datetime.now()
    return df[df["date"] == (str(now.month).rjust(2, "0")+"/"+str(now.day).rjust(2, "0"))]
a = list(globals().items())
translate = dict(zip("qwertyuiopasdfghjklzxcvbnm", "йцукенгшщзфывапролдячсмить"))
for key, value in a:
    if type(value) == type(lambda:0):
        exec("".join(map(lambda c:translate[c] if c in translate else c, key))+f" = {key}")

In [15]:
auditories = cabs - set(now(df)["classroom"])

# def free_auditories():
#     auditories = []
#     for i in cabs:
#         auditories.append(i)
#     print(*auditories)
#     return auditories

# auditories = free_auditories()

with open("file.txt",'w',encoding="utf-8") as f:
    for cab in auditories:
        f.write(cab+" ")
    f.close()
# set(now(df)["classroom"])

auditories

{'C1', 'C10', 'C3', 'C4', 'C7', 'C8', 'C9', 'CH', 'LAB2', 'MR'}